In [9]:
!pip install catboost

In [10]:
import pandas as pd
import numpy as np

data_test = pd.read_parquet('test.parquet')
submission = pd.read_csv('sample_submission.csv')

In [11]:
from datetime import date, datetime

def extract_features(row):
  dates = row['dates']
  values = row['values']

  features = {
      'start_date': int(datetime(dates[0].year, dates[0].month, dates[0].day).timestamp()),
      'end_date': int(datetime(dates[-1].year, dates[-1].month, dates[-1].day).timestamp()),
      'days_streak': (dates[-1] - dates[0]).days,
      'mean_value': np.mean(values),
      'std_value': np.std(values),
      'sum_value': np.sum(values),
      'min_value': np.min(values),
      'max_value': np.max(values)
      }
  return pd.Series(features)

def extract_columns(data):
  return data.apply(extract_features, axis=1)

In [12]:
from catboost import CatBoostClassifier
import joblib

scaler_filename = "scaler.save"
scaler = joblib.load(scaler_filename)

data_test = pd.read_parquet('test.parquet')

model = CatBoostClassifier()
model.load_model('model.cbm')

modified_test = extract_columns(data_test)
modified_test[modified_test.columns] = scaler.transform(modified_test[modified_test.columns])
y_pred = model.predict_proba(modified_test)

submission['score'] = y_pred
submission.to_csv('submission.csv', index=False)